In [ ]:
using QuantumCollocation
using NamedTrajectories
using TrajectoryIndexingUtils

using LinearAlgebra
using CairoMakie

There are no namespaces in Julia because of multiple dispatch (polymorphism). However, if you want to, you write `import QuantumCollocation as QC` then call every method using `QC.method_name`.

# Background

The model is a single qubit with X and Y control.

# Compute

In [ ]:
# I use a dictionary to store all the control problems I create
probs = Dict()

`GATES` is a utility in QuantumCollocation's _quantum_utils.jl_ that has the Paulis.

In [ ]:
H_drift = zeros(2, 2)
H_drives = [GATES[:X], GATES[:Y]]
X_goal = GATES[:X]
# Alternatively, we could go for the √X gate
SX_goal = sqrt(GATES[:X])
T = 50
Δt = .2

# X gate
# Position arguments followed by keyword arguments, separated by ;
# Currently, we use hessian_approximation because hessians might be unimplemented in certain areas
probs["X"] = UnitarySmoothPulseProblem(
    H_drift, 
    H_drives, 
    X_goal, 
    T, 
    Δt;
    hessian_approximation=true,
)

The next call will solve the problem we created, and a bunch of prints will happen.

[Reading the IPOPT output](https://coin-or.github.io/Ipopt/OUTPUT.html)

In [ ]:
solve!(probs["X"]; max_iter=100)

How to inspect an object: Where is the data?

In [ ]:
# Check type (use pipe operator to print)
typeof(probs["X"]) |> println

# Check fields
fieldnames(typeof(probs["X"])) |> println

# Data is in a NamedTrajectory
typeof(probs["X"].trajectory) |> println

# Check fields
fieldnames(typeof(probs["X"].trajectory)) |> println

# Check components of state and control
probs["X"].trajectory.components |> println

# Access data conveniently without indexing
data1 = probs["X"].trajectory[:a]

# Alternative inconvenient indexing
data2 = probs["X"].trajectory.data[probs["X"].trajectory.components[:a], :]
(data1 == data2) |> println

It's not so easy to find all the methods you might be interested in using. You have to get to know the repositories a bit. But you can use `methods` to see what arguments each method can take.

In [ ]:
methods(unitary_fidelity)

In [ ]:
unitary_fidelity(probs["X"])

In [ ]:
# convenient function for inspecting result
plot(probs["X"].trajectory)